In [1]:
import pandas as pd 
import psycopg2
from dotenv import load_dotenv
load_dotenv()
import os

DB_NAME = os.getenv("DBNAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")

In [2]:
def connect_to_db(query):
    try: 
        print("Connecting to PostgreSQL database...")
        connection = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER, 
            password=DB_PASSWORD, 
            host=DB_HOST, 
            port=DB_PORT
        )
        cursor = connection.cursor()
        print("PostgreSQL database successfully connected.")
        print(f"Executing query: {query}")
        cursor.execute(query)
        data = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        print(f"Columns: {columns}")
        df = pd.DataFrame(data, columns=columns)
        print(f"Dataframe shape: {df.shape}")
        connection.close()
        return df
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error connecting to Postgres database", error)

In [3]:
review_df = connect_to_db("SELECT * FROM review_data")
meta_df = connect_to_db("SELECT * FROM meta_data")



Connecting to PostgreSQL database...
PostgreSQL database successfully connected.
Executing query: SELECT * FROM review_data
Columns: ['rating', 'title', 'review_text', 'images', 'asin', 'parent_asin', 'user_id', 'review_timestamp', 'verified_purchase', 'helpful_vote', 'filename']
Dataframe shape: (2647447, 11)
Connecting to PostgreSQL database...
PostgreSQL database successfully connected.
Executing query: SELECT * FROM meta_data
Columns: ['id', 'filename', 'main_category', 'title', 'average_rating', 'rating_number', 'features', 'description', 'price', 'images', 'videos', 'store', 'categories', 'details', 'parent_asin', 'bought_together', 'created_at']
Dataframe shape: (1300847, 17)


In [4]:
meta_df.isna().sum()

id                 0
filename           0
main_category      0
title              0
average_rating     0
rating_number      0
features           0
description        0
price              0
images             0
videos             0
store              0
categories         0
details            0
parent_asin        0
bought_together    0
created_at         0
dtype: int64

In [4]:
df = pd.merge(review_df, meta_df, 
              how='left', 
              suffixes=['_review', '_meta'], 
              left_on=['parent_asin', 'filename'], 
              right_on=['parent_asin', 'filename'])

In [49]:
print(f'Number of Null Values: \n{df.isna().sum()}')
print(f'Number of Duplicate Values: {df.duplicated().sum()}')

Number of Null Values: 
rating                    0
title_review              0
review_text               0
images_review             0
asin                      0
parent_asin               0
user_id                   0
review_timestamp          0
verified_purchase         0
helpful_vote              0
filename                  0
id                   157836
main_category        157836
title_meta           157836
average_rating       157836
rating_number        157836
features             157836
description          157836
price                157836
images_meta          157836
videos               157836
store                157836
categories           157836
details              157836
bought_together      157836
created_at           157836
dtype: int64
Number of Duplicate Values: 6459


In [5]:
print("Complete duplicate rows:", df.duplicated().sum())

print("Duplicates on (user_id, parent_asin):", df.duplicated(subset=['user_id', 'parent_asin']).sum())

duplicates = df[df.duplicated(keep=False)]
print(f"Sample duplicates:\n{duplicates[['user_id', 'parent_asin', 'rating', 'title_review']].head(10)}")

Complete duplicate rows: 6459
Duplicates on (user_id, parent_asin): 20494
Sample duplicates:
                           user_id parent_asin  rating  \
3723  AHLDHIVNLWLVRDISEB76TEMLOG2Q  B00WDKPW9O     5.0   
3724  AHLDHIVNLWLVRDISEB76TEMLOG2Q  B07VQZPHHF     5.0   
3725  AHLDHIVNLWLVRDISEB76TEMLOG2Q  B00WDKPW9O     5.0   
3726  AHLDHIVNLWLVRDISEB76TEMLOG2Q  B07VQZPHHF     5.0   
3875  AEKMLJQPUBHCKDDHKVC74DYKON6Q  B09NBJJCM6     5.0   
3876  AEKMLJQPUBHCKDDHKVC74DYKON6Q  B09NBJJCM6     5.0   
4338  AEKDWV2NZYGECRICKDZ2ANLSKUAA  B0B8Q3WTGP     4.0   
4342  AEKDWV2NZYGECRICKDZ2ANLSKUAA  B0B8Q3WTGP     4.0   
4499  AG2XZ2N3OGKGELCALULKCA43EY4Q  B083XH4FYP     5.0   
4503  AG2XZ2N3OGKGELCALULKCA43EY4Q  B083XH4FYP     5.0   

                                        title_review  
3723                           Moisturizing  ur skin  
3724                           Useful For Showering.  
3725                           Moisturizing  ur skin  
3726                           Useful For Shower

In [6]:
df = df.drop_duplicates()

remaining_user_product_dupes = df.duplicated(subset=['user_id', 'parent_asin']).sum()
print(f"After removing exact duplicates: {remaining_user_product_dupes} user-product duplicates remain")


if remaining_user_product_dupes > 0:
    dupes = df[df.duplicated(subset=['user_id', 'parent_asin'], keep=False)]
    dupes['date_time'] = pd.to_datetime(dupes['review_timestamp'], unit='ms')
    print("Sample remaining duplicates:")
    print(dupes[['user_id', 'parent_asin', 'date_time', 'rating', 'title_review']].head(10))

After removing exact duplicates: 14035 user-product duplicates remain
Sample remaining duplicates:
                          user_id parent_asin               date_time  rating  \
70   AHV6QCNBJNSGLATP56JAWJ3C4G2A  B0B2L218H2 2020-11-20 15:25:19.061     4.0   
71   AHV6QCNBJNSGLATP56JAWJ3C4G2A  B0B2L218H2 2020-11-19 19:33:29.168     4.0   
190  AFXF3EGQTQDXMRLDWFU7UBFQZB7Q  B08HGZXLP6 2021-01-23 00:07:32.090     5.0   
193  AFXF3EGQTQDXMRLDWFU7UBFQZB7Q  B08HGZXLP6 2021-01-20 00:34:13.719     5.0   
198  AFQQQ5LGNSQUEBGDCYBAZZE5T3DA  B08WCG4JCC 2020-09-14 21:57:49.811     5.0   
253  AFQQQ5LGNSQUEBGDCYBAZZE5T3DA  B08WCG4JCC 2020-09-09 16:50:55.886     5.0   
435  AHU2GG5RF6YAEWUFNLH3QH5RHDNQ  B01DD1NOZU 2021-11-26 18:52:21.170     5.0   
436  AHU2GG5RF6YAEWUFNLH3QH5RHDNQ  B01DD1NOZU 2021-11-26 18:38:38.416     5.0   
760  AEZP6Z2C5AVQDZAJECQYZWQRNG3Q  B08YJV9YL2 2020-11-11 18:04:54.655     4.0   
766  AEZP6Z2C5AVQDZAJECQYZWQRNG3Q  B08YJV9YL2 2020-11-01 15:13:41.231     4.0   

         

C:\Users\danib\AppData\Local\Temp\ipykernel_9064\163561704.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupes['date_time'] = pd.to_datetime(dupes['review_timestamp'], unit='ms')


In [7]:
df = df.dropna(subset=['main_category'])
print(df.isna().sum())

rating               0
title_review         0
review_text          0
images_review        0
asin                 0
parent_asin          0
user_id              0
review_timestamp     0
verified_purchase    0
helpful_vote         0
filename             0
id                   0
main_category        0
title_meta           0
average_rating       0
rating_number        0
features             0
description          0
price                0
images_meta          0
videos               0
store                0
categories           0
details              0
bought_together      0
created_at           0
dtype: int64


# **Data Cleaning**

In [8]:
df.columns

Index(['rating', 'title_review', 'review_text', 'images_review', 'asin',
       'parent_asin', 'user_id', 'review_timestamp', 'verified_purchase',
       'helpful_vote', 'filename', 'id', 'main_category', 'title_meta',
       'average_rating', 'rating_number', 'features', 'description', 'price',
       'images_meta', 'videos', 'store', 'categories', 'details',
       'bought_together', 'created_at'],
      dtype='object')

In [9]:
df['review_date'] = pd.to_datetime(df['review_timestamp'], unit='ms')
df['review_year'] = df['review_date'].dt.year

df['review_text'] = df['review_text'].fillna('').astype(str)
df['title_review'] = df['title_review'].fillna('').astype(str)
print(f"The dates range from {df['review_date'].min()} to {df['review_date'].max()}")

The dates range from 2020-01-01 08:00:00.770000 to 2023-09-06 22:41:39.945000


In [10]:
df['rating'] = df['rating'].astype(float)
df['average_rating'] = pd.to_numeric(df['average_rating'], errors='coerce')
df['price'] = pd.to_numeric(df['price'], errors='coerce') 

df['helpful_vote'] = df['helpful_vote'].astype(int)
df['rating_number'] = pd.to_numeric(df['rating_number'], errors='coerce').astype('Int64')  
df['id'] = df['id'].astype('Int64')  


df['verified_purchase'] = df['verified_purchase'].astype(bool)

mask = (df['main_category'] == '') | (df['main_category'].isnull())

for filename in df.loc[mask, 'filename'].unique():
    valid_categories = df[
        (df['filename'] == filename) & 
        (df['main_category'] != '') & 
        (df['main_category'].notna())
    ]['main_category'].unique()
    
    if len(valid_categories) > 0:
        correct_category = df[
            (df['filename'] == filename) & 
            (df['main_category'] != '') & 
            (df['main_category'].notna())
        ]['main_category'].mode().iloc[0]
        
        df.loc[
            (df['filename'] == filename) & 
            ((df['main_category'] == '') | (df['main_category'].isnull())),
            'main_category'
        ] = correct_category
        
        print(f"Fixed {filename}: {correct_category}")
    else:
        print(f"Warning: No valid main_category found for {filename}")

Fixed Automotive.jsonl: Automotive
Fixed Appliances.jsonl: Tools & Home Improvement
Fixed Movies_and_TV.jsonl: Prime Video
Fixed Arts_Crafts_and_Sewing.jsonl: Arts, Crafts & Sewing
Fixed Baby_Products.jsonl: Baby
Fixed Beauty_and_Personal_Care.jsonl: All Beauty
Fixed Books.jsonl: Books
Fixed CDs_and_Vinyl.jsonl: Digital Music
Fixed Cell_Phones_and_Accessories.jsonl: Cell Phones & Accessories
Fixed Gift_Cards.jsonl: Gift Cards
Fixed Clothing_Shoes_and_Jewelry.jsonl: AMAZON FASHION
Fixed Electronics.jsonl: All Electronics
Fixed Grocery_and_Gourmet_Food.jsonl: Grocery
Fixed Health_and_Household.jsonl: Health & Personal Care
Fixed Handmade_Products.jsonl: Handmade
Fixed Home_and_Kitchen.jsonl: Amazon Home
Fixed Kindle_Store.jsonl: Buy a Kindle
Fixed Industrial_and_Scientific.jsonl: Industrial & Scientific
Fixed Musical_Instruments.jsonl: Musical Instruments
Fixed Office_Products.jsonl: Office Products
Fixed Patio_Lawn_and_Garden.jsonl: Amazon Home
Fixed Pet_Supplies.jsonl: Pet Supplies
Fix

# **Data Inspection**

In [11]:
print(f"Number of rows: {df.shape[0]:,}")
print(f"Number of unique users: {len(df['user_id'].unique()):,}")
print(f"Number of unique products: {len(df['parent_asin'].unique()):,}")
print(f"Number of unique categories: {len(df['main_category'].unique()) if 'main_category' in df.columns else 'N/A'}")
print(f"Price coverage: {(df['price'].notna().sum() / len(df) * 100):.1f}%")
print(f"Avg rating: {df['rating'].mean():.2f}")

total_purchases_per_user = df.groupby('user_id')['parent_asin'].count()
avg_total_purchases = total_purchases_per_user.mean()
print(f"Average total purchases per user: {avg_total_purchases:.2f}")
print("-" * 50)  

Number of rows: 2,483,400
Number of unique users: 316,045
Number of unique products: 1,300,847
Number of unique categories: 48
Price coverage: 72.8%
Avg rating: 4.32
Average total purchases per user: 7.86
--------------------------------------------------


In [12]:
sorted(df['main_category'].unique())

['',
 'AMAZON FASHION',
 'All Beauty',
 'All Electronics',
 'Amazon Devices',
 'Amazon Fire TV',
 'Amazon Home',
 'Apple Products',
 'Appliances',
 'Appstore for Android',
 'Arts, Crafts & Sewing',
 'Audible Audiobooks',
 'Automotive',
 'Baby',
 'Books',
 'Buy a Kindle',
 'Camera & Photo',
 'Car Electronics',
 'Cell Phones & Accessories',
 'Collectible Coins',
 'Collectibles & Fine Art',
 'Computers',
 'Digital Music',
 'Entertainment',
 'Fine Art',
 'Fire Phone',
 'GPS & Navigation',
 'Gift Cards',
 'Grocery',
 'Handmade',
 'Health & Personal Care',
 'Home Audio & Theater',
 'Industrial & Scientific',
 'Magazine Subscriptions',
 'Movies & TV',
 'Musical Instruments',
 'Office Products',
 'Pet Supplies',
 'Portable Audio & Accessories',
 'Premium Beauty',
 'Prime Video',
 'SUBSCRIPTION BOXES',
 'Software',
 'Sports & Outdoors',
 'Sports Collectibles',
 'Tools & Home Improvement',
 'Toys & Games',
 'Video Games']

# **Recommendation System Suitability**

In [17]:
def assess_rec_sys_readiness(data): 
    n_users = data['user_id'].nunique()
    n_items = data['parent_asin'].nunique()
    n_interactions = len(data)
    sparsity = (1-(n_interactions/(n_users*n_items))) * 100
    
    print(f"Number of users: {n_users:,}")
    print(f"Number of items: {n_items:,}")
    print(f"Number of interactions: {n_interactions:,}")
    print(f"Sparsity: {sparsity:.2f}%")
    
    user_count = data['user_id'].value_counts()
    item_count = data['parent_asin'].value_counts()
    
    print(f" Users with 1 review: {(user_count==1).sum():,} ({(user_count==1).sum()/n_users*100:.1f}%)")
    print(f"  Items with 1 review: {(item_count == 1).sum():,} ({(item_count == 1).sum()/n_items*100:.1f}%)")
    print(f"  Users with 5+ reviews: {(user_count >= 5).sum():,}")
    print(f"  Items with 5+ reviews: {(item_count >= 5).sum():,}")
    
    
    if sparsity > 99.5: 
        print("Very sparse - Consider content based + hybrid approaches")
    elif sparsity > 99:
        print("Moderately sparse - Hybrid approach recommended")  
    else:
        print("Good density - Collaborative filtering viable")
        

In [18]:
assess_rec_sys_readiness(df)

Number of users: 316,045
Number of items: 1,300,847
Number of interactions: 2,483,400
Sparsity: 100.00%
 Users with 1 review: 178,719 (56.5%)
  Items with 1 review: 934,967 (71.9%)
  Users with 5+ reviews: 73,206
  Items with 5+ reviews: 71,614
Very sparse - Consider content based + hybrid approaches


In [ ]:
print(f"Saving to Parquet...")
df.to_parquet("amazon_df.parquet", index=False)
print(f"Done!")